In [10]:
# important packages to import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys, os

from sklearn import datasets
from sklearn import preprocessing, cross_validation, svm
from sklearn.cross_validation import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

from datetime import datetime, timedelta
from collections import defaultdict

# DataFrame Summary and inital cleaning

In [3]:
# holistic summary of the given data set. 
# "remove_bad_rowCol" can be turned on to remove non-informative col / row
def holistic_summary(df, remove_bad_rowCol = False, verbose = True):
    # remove non-informative columns
    if(remove_bad_rowCol):
        df = df.drop(df.columns[df.isnull().sum() >= .9 * len(df)], axis = 1)
        df = df.drop(df.index[df.isnull().sum(axis = 1) >= .5* len(df.columns)], axis = 0)
        
    # fix column names:
    df.columns = [c.replace(" ", "_").lower() for c in df.columns]
    
    print('***************************************************************')
    print('Begin holistic summary: ')
    print('***************************************************************\n')
    
    print('Dimension of df: ' + str(df.shape))
    print('Percentage of good observations: ' + str(1 - df.isnull().any(axis = 1).sum()/len(df)))
    print('---------------------------------------------------------------\n')
    
    print("Rows with nan values: " + str(df.isnull().any(axis = 1).sum()))
    print("Cols with nan values: " + str(df.isnull().any(axis = 0).sum()))
    print('Breakdown:')
    print(df.isnull().sum()[df.isnull().sum()!=0])
    print('---------------------------------------------------------------\n')
    
    print('Columns details: ')
    print('Columns with known dtypes: ')
    good_cols = pd.DataFrame(df.dtypes[df.dtypes!='object'], columns = ['type'])
    good_cols['nan_num'] = [df[col].isnull().sum() for col in good_cols.index]
    good_cols['unique_val'] = [df[col].nunique() for col in good_cols.index]
    good_cols['example'] = [df[col][1] for col in good_cols.index]
    good_cols = good_cols.reindex(good_cols['type'].astype(str).str.len().sort_values().index)
    print(good_cols)
    print('\n')
    
    try:
        print('Columns with unknown dtypes:')
        bad_cols = pd.DataFrame(df.dtypes[df.dtypes=='object'], columns = ['type'])
        bad_cols['nan_num'] = [df[col].isnull().sum() for col in bad_cols.index]
        bad_cols['unique_val'] = [df[col].nunique() for col in bad_cols.index]
        bad_cols['example(sliced)'] = [str(df[col][1])[:10] for col in bad_cols.index]
        bad_cols = bad_cols.reindex(bad_cols['example(sliced)'].str.len().sort_values().index)
        print(bad_cols)
    except Exception as e:
        print('No columns with unknown dtypes!')
    print('_______________________________________________________________\n\n\n')
    #if not verbose: enablePrint()
    return df

In [22]:
#testing
flights = pd.read_csv('../Datasets/flights.csv')
flights = holistic_summary(boston, remove_bad_rowCol = True, verbose = True)

***************************************************************
Begin holistic summary: 
***************************************************************

Dimension of df: (336776, 20)
Percentage of good observations: 0.971999192341
---------------------------------------------------------------

Rows with nan values: 9430
Cols with nan values: 6
Breakdown:
dep_time     8255
dep_delay    8255
arr_time     8713
arr_delay    9430
tailnum      2512
air_time     9430
dtype: int64
---------------------------------------------------------------

Columns details: 
Columns with known dtypes: 
                   type  nan_num  unique_val  example
unnamed:_0        int64        0      336776      2.0
year              int64        0           1   2013.0
month             int64        0          12      1.0
day               int64        0          31      1.0
sched_dep_time    int64        0        1021    529.0
sched_arr_time    int64        0        1163    830.0
flight            int64        

# DataFrame cleaning and type fixing

In [23]:
# fixing dtypes: time and numeric variables
def fix_dtypes(df, time_cols, num_cols):
    
    print('***************************************************************')
    print('Begin fixing data types: ')
    print('***************************************************************\n')
    
    def fix_time_col(df, time_cols):
        for time_col in time_cols:
            df[time_col] = pd.to_datetime(df[time_col], errors = 'coerce')
        print('---------------------------------------------------------------')
        print('The following time columns has been fixed: ')
        print(time_cols)
        print('---------------------------------------------------------------\n')

    def fix_factor_col(df):
        categorical_col = []
        for col in df.columns:
            if (df[col].nunique()<12) & (df[col].nunique()<len(df)/10):
                df[col] = df[col].astype('category')
                categorical_col.append(col)
        print('---------------------------------------------------------------')
        print('The following category columns has been fixed: ')
        print(categorical_col)
        print('---------------------------------------------------------------\n')

    def fix_num_col(df, num_cols):
        for col in num_cols:
            df[col] = pd.to_numeric(df[col], errors = 'coerce')
        print('---------------------------------------------------------------')
        print('The following number columns has been fixed: ')
        print(num_cols)
        print('---------------------------------------------------------------\n')
        
    if(len(num_cols) > 0):
        fix_num_col(df, num_cols)
    fix_time_col(df, time_cols)
    #fix_factor_col(df)
    #fix_string_col(df)
    print('---------------------------------------------------------------')
    print('Final data types:')
    result = pd.DataFrame(df.dtypes, columns = ['type'])
    result = result.reindex(result['type'].astype(str).str.len().sort_values().index)
    print(result)
    print('_______________________________________________________________\n\n\n')
    return df

# fix nan values 
def fix_nan(df, how = 'partial'):
    
    print('***************************************************************')
    print('Begin fixing nans: ')
    print('***************************************************************\n')
    if(how == 'partial'):     
        print('The following columns are removed due to nan1: ')
        print(df.columns[df.isnull().sum() >= .5 * len(df)])
        df = df.drop(df.columns[df.isnull().sum() >= .5 * len(df)], axis = 1)
    elif(how == 'all'):
        print('The following columns are removed due to nan2: ')
        print(df.columns[df.isnull().sum() == len(df)])
        df = df.drop(df.columns[df.isnull().sum() == len(df)], axis = 1)
    print('_______________________________________________________________\n\n\n')
    return df

In [26]:
# testing
flights = fix_nan(flights, 'partial')
flights = fix_dtypes(flights, time_cols = ['time_hour'], num_cols = [])

***************************************************************
Begin fixing nans: 
***************************************************************

The following columns are removed due to nan1: 
Index([], dtype='object')
_______________________________________________________________



***************************************************************
Begin fixing data types: 
***************************************************************

---------------------------------------------------------------
The following time columns has been fixed: 
['time_hour']
---------------------------------------------------------------

---------------------------------------------------------------
Final data types:
                          type
unnamed:_0               int64
year                     int64
month                    int64
day                      int64
hour                     int64
sched_dep_time           int64
distance                 int64
sched_arr_time           int64
minute 

# Machine learning toolkit

In [5]:
def k_fold_cross_validation (clf, X_train, y_train, verbose = True, k = 5):
    clf.fit(X_train, y_train) # fit the model
    cv = KFold(X_train.shape[0], 5, shuffle = True)
    scores = cross_val_score(clf, X_train, y_train, cv = cv) # 5-fold cross_validation
    
    if verbose == True: #if we want to print status
        print("Coefficient of determination on training set:", clf.score(X_train, y_train))
        print("Average coefficient of determination using 5-fold crossvalidation:", np.mean(scores))
        print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    # returns the mean score and 2 sigma
    return np.mean(scores), scores.std() * 2

def label_encode(df, col_encode):
    encoder = preprocessing.LabelEncoder()
    for col in col_encode:
        df[col] = encoder.fit_transform(df[col])
    return df